In [132]:
import json
import requests
import six
import six.moves.urllib.parse as urllibparse
import pandas as pd
import matplotlib
import pyperclip as cp
import heapq
import base64

%matplotlib inline
import matplotlib.pyplot as plt

In [51]:
def get_auth_code(url):
    start = url.index('?')
    end = url.index('&')
    code = url[start + 6:end]
    return code

def create_auth_header(client_id, client_secret):
    auth_header = base64.b64encode(six.text_type(client_id + ':' + client_secret).encode('ascii'))
    return {'Authorization': 'Basic %s' % auth_header.decode('ascii')}

In [52]:
username = 'deeaw2ybwljjvzzv7sftosnqi'
scope = "user-top-read user-read-recently-played user-library-read user-library-modify user-read-playback-state user-read-currently-playing user-modify-playback-state"
client_id = 'e88d92c779fb4f2baf5f69a919db89ec'
client_secret = 'e27b14511a554ea58a2b42d3b6ef5b15'
redirect_uri = 'https://example.com/callback/'
state = 'ilovetocodelol'

auth_code_url = "https://accounts.spotify.com/authorize"
auth_code_dict = {"client_id": client_id, "response_type": "code", "redirect_uri": redirect_uri, "state": state, "scope": scope, "show_dialog": "false"}

auth_code_params = urllibparse.urlencode(auth_code_dict)
                    
get_response = requests.get(auth_code_url, params=auth_code_params)
if get_response.status_code != 200:
    print("Error:", get_response.reason)
    
print(get_response.url)

https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fclient_id%3De88d92c779fb4f2baf5f69a919db89ec%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fexample.com%252Fcallback%252F%26state%3Dilovetocodelol%26scope%3Duser-top-read%2Buser-read-recently-played%2Buser-library-read%2Buser-library-modify%2Buser-read-playback-state%2Buser-read-currently-playing%2Buser-modify-playback-state%26show_dialog%3Dfalse


In [54]:
token_url = "https://accounts.spotify.com/api/token"
get_response_url = "https://example.com/callback/?code=AQCCf75fqJ9hrDKTvjBgm3WrWuBF_owAMgsuLF0dEo7YbG4I46XU5XoQQHzI01uXe_uo2W1IcLhoOELTSy0yx-exkTS-Swqz3ICOoWoTyn_s_PZZdgSmwVJom5yxO6a2Zdk8ldB5HiUvvwc7SzfWA6_9UVEyO5n_v8_MpDHhFDk2gqXm0w6ukoPI5Rkih8sW9-fr8n375_xr82h3sAVpMnkd-4zLIVLlE0485loTiMeoUCzQliLqisPadnPrM6FvmM3UlUlM-akiRKGDz-2_OmQVctSoD755HjO2cd0emLKgu4zotCsr_c5mepXfa9sBFhGO33fNk5EiK8etn-HcW0yZqYpMIvIR9lIEUHT-2AjgpQxm2Xkwu5o6J4EBU0mm9eJRnxZwcdMwRxLE0-zcRhzbeVCtnxAcD3c&state=ilovetocodelol"
token_data = {"redirect_uri": redirect_uri, "code": get_auth_code(get_response_url), "grant_type": "authorization_code", "scope": scope}
token_headers = create_auth_header(client_id, client_secret)

post_response = requests.post(token_url, data=token_data, headers=token_headers)

if post_response.status_code != 200:
    print("Error:", post_response.reason)

ACCESS_TOKEN = json.loads(post_response.text)["access_token"]
print(ACCESS_TOKEN)

BQBIH630pFnDb4bPMIBPlCspYdSqqANSF6irT6Sz6GoELlOW4P40f70_APk7_GClfhS7GLwm3KGd5hxmKZBiWKNWP5IaUnIK3wmayDzidRogzxPehguy-t6ZHhuIXyihUzLfy1E9MsZ7G8NV7ujgDz9tnnnFFEYCDNAZ6CzXJls1ZKiPd2qMQzmfkXIjxIUzYehi3ur9Lg


In [55]:
url = "https://api.spotify.com/v1/me/tracks"
headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
tracks = requests.get(url, headers=headers)
parsed = json.loads(tracks.text)
cp.copy(tracks.text)

count_songs = parsed["total"]
print("Number of songs: {}".format(count_songs))

Number of songs: 822


In [56]:
all_songs = []

for i in range(int(ceil(count_songs/50.0))):
    offset = 50*i
    url = "https://api.spotify.com/v1/me/tracks?limit=50&offset={}".format(offset)
    track_headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
    tracks = requests.get(url, headers=track_headers)
    cp.copy(tracks.text)
    parsed = json.loads(tracks.text)

    all_songs.extend(parsed['items'])
    
try:
    if (len(all_songs) == count_songs):
        print("No responses failed.")
except:
    print("Response from API failed.")
    
print("Number of gathered songs: {}".format(len(all_songs)))

No responses failed.
Number of gathered songs: 822


In [57]:
songs_all = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs))]
songs_under_5 = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs)) if all_songs[i]['track']['duration_ms'] <= 300000]    
album_ids = {(all_songs[i]['track']['album']['name'], all_songs[i]['track']['album']['id']) 
             for i in range(len(all_songs))}

print("Total songs:", len(songs_all))
print("Total songs under 5 minutes:", len(songs_under_5))

Total songs: 822
Total songs under 5 minutes: 737


In [80]:
largest = heapq.nsmallest(10, songs_under_5)
largest = [(key, value) for value, key in largest]
for song in largest:
    print(song)

('Flyaway', 0)
('High for This', 0)
('Might Not', 0)
('Nonchalant', 0)
('Starboy', 0)
('What You Need', 0)
('10AM/Save The World (feat. Gucci Mane)', 1)
('Lesbian (feat. Gunna & Young Thug)', 1)
('Only 1 (Interlude) (with Travis Scott)', 1)
('Up To Something (feat. Travis Scott & Young Thug)', 1)


In [85]:
count = 0
for item in album_ids:
    if (count < 10):
        print(item)
        count += 1
    else:
        break

('ASTROWORLD', '41GuZcammIkupMPKH2OJ6I')
('!', '4YEhTvqsptB5D1n095QL5r')
('Snow White', '6tvVlwo0u8wBCmRvLCJIXQ')
('Suncity', '4UNwL1B7JoymNUiTFJa52B')
('Blank Face LP', '0YbpATCIng8Fz2JrfHmEf7')
('DAMN.', '4eLPsYPBmXABThSJ821sqY')
('Una Mattina', '4aSt14A98gEpCUpKecrl4V')
('Walking On A Dream', '04gYcIojJt78nYnN5oOrKt')
('Black Panther The Album Music From And Inspired By', '3pLdWdkj83EYfDN6H2N8MR')
('Marie Curie', '4m99RhYGgvz4CYOtVtXTaz')


In [95]:
from __future__ import division

def get_top_music(type="artists", limit=20, time_range="medium_term", access_token=ACCESS_TOKEN):
    url = "https://api.spotify.com/v1/me/top/"
    if (type == "artists"):
        url += "artists"
    else:
        url += "tracks"
    
    query_params={"limit": str(limit), "offset": "0", "time_range": time_range}
    headers = {'Authorization': "Bearer {}".format(access_token)}
    
    top_tracks_and_artists = requests.get(url=url, params=query_params, headers=headers)
    parsed = json.loads(top_tracks_and_artists.text)
    items = parsed['items']

    top_music = []
    
    if (type == "artists"):
        for i in range(len(items)):
            top_music.append(["Rank: {}".format(str(i+1)), items[i]['name'], int(items[i]['followers']['total']), items[i]['images'][0]['url']])
    else:
        for i in range(len(items)):
            current_item = items[i]
            artists = current_item['artists']
            current_album = current_item['album']
            length_of_song = str((current_item['duration_ms'] // 1000) // 60) + "m " + str((current_item['duration_ms'] // 1000) % 60) + "s"
            top_music.append(["Rank: {}".format(str(i+1)), current_item['name'], 
                               [artists[j]['name'] for j in range(len(artists))], 
                               current_album['name'], length_of_song, current_album['release_date']])
    return top_music
    

In [128]:
def get_top_genres(artist_limit=50, genre_limit=10, time_range="medium_term", access_token=ACCESS_TOKEN):
    url = "https://api.spotify.com/v1/me/top/artists"
    all_genres = {}
    
    query_params={"limit": str(artist_limit), "offset": "0", "time_range": time_range}
    headers = {'Authorization': "Bearer {}".format(access_token)}
    
    top_tracks_and_artists = requests.get(url=url, params=query_params, headers=headers)
    parsed = json.loads(top_tracks_and_artists.text)
    items = parsed['items']
    
    for i in range(len(items)):
        genres = items[i]['genres']
        for genre in genres:
            if genre not in all_genres.keys():
                all_genres[genre] = 1
            else:
                all_genres[genre] += 1
                
    heap = [[value, key.upper()] for key, value in all_genres.items()]
    largest = heapq.nlargest(genre_limit, heap)
        
    return largest

In [129]:
for artist in get_top_music(type="artists", limit=15, time_range="long_term"):
    print(artist)

['Rank: 1', 'Drake', 39483151, 'https://i.scdn.co/image/012ecd119617ac24ab56620ace4b81735b172686']
['Rank: 2', 'Tory Lanez', 2422139, 'https://i.scdn.co/image/319b5b30148cb0614be70f115b272afc18ea69c7']
['Rank: 3', 'Playboi Carti', 2572889, 'https://i.scdn.co/image/d3c4c2a06332c88e967387b38177d17d20e2331b']
['Rank: 4', 'NAV', 1501093, 'https://i.scdn.co/image/819c7e7c7126e367b35ff23e0653b1779069128f']
['Rank: 5', 'The Weeknd', 16751740, 'https://i.scdn.co/image/a5fa0d35a5070867261d100b879cac028fe96775']
['Rank: 6', '6LACK', 1713920, 'https://i.scdn.co/image/b7b33aaeac1367b35d9c74a53fc9cfbd38f79fee']
['Rank: 7', 'Travis Scott', 6219496, 'https://i.scdn.co/image/dc5eba5e032c2e5bc4d42c89d61ffee5875c981f']
['Rank: 8', 'A$AP Rocky', 5723326, 'https://i.scdn.co/image/cce607c4d975ae49d68b484eaef51a6413fa73b1']
['Rank: 9', 'Khalid', 7774624, 'https://i.scdn.co/image/e52c824585bfeef4d674b509437120805300f557']
['Rank: 10', 'Future', 7957717, 'https://i.scdn.co/image/d5ac5dc59664cf7c64795271602316

In [130]:
for track in get_top_music(type="tracks", limit=15, time_range="long_term"):
    print(track)

['Rank: 1', 'Let It Go', ['Playboi Carti'], 'Playboi Carti', '2m 30s', '2017-04-14']
['Rank: 2', 'FlatBed Freestyle', ['Playboi Carti'], 'Die Lit', '3m 13s', '2018-05-11']
['Rank: 3', 'Call Me', ['NAV', 'Metro Boomin'], 'Perfect Timing', '3m 35s', '2017-07-21']
['Rank: 4', 'STARGAZING', ['Travis Scott'], 'ASTROWORLD', '4m 30s', '2018-08-03']
['Rank: 5', 'ASTROTHUNDER', ['Travis Scott'], 'ASTROWORLD', '2m 22s', '2018-08-03']
['Rank: 6', 'OTW', ['Khalid', '6LACK', 'Ty Dolla $ign'], 'OTW', '4m 23s', '2018-04-20']
['Rank: 7', 'No Time (feat. Gunna)', ['Playboi Carti', 'Gunna'], 'Die Lit', '3m 39s', '2018-05-11']
['Rank: 8', 'R.I.P. Fredo (feat. Young Nudy) - Notice Me', ['Playboi Carti', 'Young Nudy'], 'Die Lit', '2m 41s', '2018-05-11']
['Rank: 9', 'R.I.P.', ['Playboi Carti'], 'Die Lit', '3m 12s', '2018-05-11']
['Rank: 10', "Why DON'T You LOVE me?", ['Tory Lanez'], 'LoVE me NOw', '3m 32s', '2018-10-26']
['Rank: 11', 'MiAMi (feat. Gunna)', ['Tory Lanez', 'Gunna'], 'LoVE me NOw', '4m 4s', '2

In [131]:
for genre in get_top_genres(genre_limit=15, time_range="long_term"):
    print(genre)

[40, 'RAP']
[28, 'POP RAP']
[28, 'POP']
[26, 'TRAP MUSIC']
[21, 'HIP HOP']
[12, 'SOUTHERN HIP HOP']
[9, 'ATL HIP HOP']
[6, 'R&B']
[6, 'CANADIAN HIP HOP']
[4, 'UNDERGROUND HIP HOP']
[4, 'TORONTO RAP']
[3, 'VAPOR TRAP']
[3, 'TRAP SOUL']
[3, 'CONSCIOUS HIP HOP']
[3, 'CHICAGO RAP']
